Carregando os dados necessários

In [29]:
from pandas import DataFrame, read_csv, to_datetime, concat
from os import path

datadir = 'C:/Users/User/Desktop/data_science/mentoria_data_mundo/projeto_ecommerce/_data/clean'

df_clientes = read_csv(path.join(datadir, 'e_commerce_clientes.csv'))
df_carrinhos = read_csv(path.join(datadir, 'e_commerce_carrinhos.csv'))
df_pedidos = read_csv(path.join(datadir, 'e_commerce_pedidos.csv'))

Realizando os tratamentos dos dados necessários em df_pedidos

In [30]:
# Do dataframe 'df_pedidos' Iremos pegar a coluna 'total_pago' e a coluna 'id_cliente', assim teremos o valor total pago por cliente, além da data.
# A ideia é dividir em dois grupos, o ticket médio dos clientes em datas normais, e em datas especiais.

# Trocando as ',' por '.' para converter para float na coluna 'total_pago'
df_pedidos['total_pago'] = (df_pedidos['total_pago']
 .replace(',', '.', regex=True)
 .astype({'total_pago': float}))

# Trocando o tipo da coluna 'data' para datetime (formato do pandas para datas)
df_pedidos['data'] = to_datetime(df_pedidos['data'], format='%d/%m/%Y %H:%M:%S')
df_pedidos['data'] = df_pedidos['data'].apply(lambda x: x.strftime('%Y/%m/%d'))

# Filtrando apenas as colunas necessárias do dataframe 'df_pedidos'
df_pedidos = df_pedidos.filter(['id_cliente', 'total_pago', 'data'])

Criando df_pedidos com datas especiais e não especiais

In [52]:
# Vamos criar a lista com as datas especiais
datas_especiais = ([
    '2024/01/01', '2024/01/06', '2024/01/25', '2024/02/10', '2024/02/11', 
    '2024/02/12', '2024/02/13', '2024/02/14', '2024/02/24', '2024/03/08', 
    '2024/03/10', '2024/03/19', '2024/03/29', '2024/04/01', '2024/04/21', 
    '2024/04/22', '2024/04/25', '2024/04/30', '2024/05/01', '2024/05/13', 
    '2024/05/31', '2024/06/05', '2024/06/12', '2024/06/24', '2024/07/02', 
    '2024/07/09', '2024/07/20', '2024/08/11', '2024/08/15', '2024/08/19', 
    '2024/09/07', '2024/09/29', '2024/10/12', '2024/10/15', '2024/10/28', 
    '2024/11/02', '2024/11/15', '2024/11/20', '2024/12/24', '2024/12/25', '2024/12/31'])

# Agora criaremos outro dataframe apenas com as datas_especiais e um com datas normais
## Aqui temos uma amostra relativamente pequena, pois contém apenas 14 linhas das 145
df_pedidos_datas_especiais = df_pedidos[df_pedidos['data'].isin(datas_especiais)].reset_index(drop=True)

df_pedidos_datas_normais = df_pedidos[df_pedidos['data'].isin(datas_especiais) == False].reset_index(drop=True)


,id_cliente,total_pago,data
0,31d69aed10,508.90,2024/08/04
1,3e109eedd5,518.90,2024/08/01
2,3e109eedd5,518.90,2024/08/01
3,3e109eedd5,518.90,2024/08/01
4,3e109eedd5,518.90,2024/08/01
...,...,...,...
124,0db3cf4f27,14.53,2024/05/22
125,0db3cf4f27,13.50,2024/05/22
126,0db3cf4f27,139.30,2024/05/22
127,0db3cf4f27,34.90,2024/05/22
